## Импортируем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from sklearn.naive_bayes import BernoulliNB

## Опишем функции 

In [2]:
# функция поиска и удаления нулевых столбцов
def del_zero (data):
    zero_value=[]
    for i in range(1,261):
        if data.iloc[:,i].sum()==0: zero_value.append(data.iloc[:,i].name)
    print('Пустые столбцы ',zero_value, ' удалены')
    data=data.drop(zero_value, axis=1)
    return data
    
# функция поиска и удаления признаков с пропущенными значениями число которых больше порога a_del 
# функция изначально была собрана для оценки признаков с порпущенными значениями, поэтому в ее структуре 
# сохранился код для вывода значений.
def del_na(data, a_del):
    na_values=pd.DataFrame(columns=[['null_sum','null_in_0','null_in_1', 'null_in_2']])
    for i in range(0,data.shape[1]):
        temp=[]
        if data.iloc[:,i].isnull().sum()>a_del:
            temp.append(data.iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==0').iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==1').iloc[:,i].isnull().sum())
            temp.append(data.query('TARGET==2').iloc[:,i].isnull().sum())
            nm=data.iloc[:,i].name
            na_values.loc[nm]=temp
    data=data.drop(list(na_values.index), axis=1)
    print(' Признаки с количеством порпусков >',a_del, '\n', na_values, ' удалены')
    return(data)

# функция поиска и удаления признаков в которых есть сильный дисбаланс, одно из значений подавляющее 
# (больше порога max_count), из далее построенных гистограм и эксперииментов , был сделан вывод что 
# интересным может показаться порог отсечения 17000
def del_disbal(data, max_count):
    disbal_values=pd.DataFrame(columns=[['max_sum','max_in_0','max_in_1','max_in_2']])
    for i in range(0,data.shape[1]):
        temp=[]
        if data.iloc[:,i].value_counts().max()>max_count:
            temp.append(data.iloc[:,i].value_counts().max())
            temp.append(data.query('TARGET==0').iloc[:,i].value_counts().max())
            temp.append(data.query('TARGET==1').iloc[:,i].value_counts().max())
            temp.append(data.query('TARGET==2').iloc[:,i].value_counts().max()) 
            nm=data.iloc[:,i].name
            disbal_values.loc[nm]=temp
    data=data.drop(list(disbal_values.index), axis=1)
    print(' Признаки с дисбалансом >',max_count, '\n', disbal_values, ' удалены')
    return(data)

# функцию заполнения пропущенных значений, пользуемся библиотекой sklearn.impute
def imputer(data, n_n):
    imp = KNNImputer(n_neighbors=n_n, weights="uniform")
    not_nan_mass=imp.fit_transform(data)
    ft_list=list(data.columns)
    data_not_nan=pd.DataFrame(not_nan_mass, columns=ft_list)
    return(data_not_nan)

# ВСПОМОГАТЕЛЬНАЯ ФУНКЦИЯ посмотрим на признаки с пропущенными значениями и их представленность в каждом классе
def na_val(data):
    na_values=pd.DataFrame(columns=['null_sum'])
    for i in range(0,data.shape[1]):
        temp=[]
        if data.iloc[:,i].isnull().sum()>0:
            temp.append(data.iloc[:,i].isnull().sum())
            #temp.append(data.query('TARGET==0').iloc[:,i].isnull().sum())
            #temp.append(data.query('TARGET==1').iloc[:,i].isnull().sum())
            #temp.append(data.query('TARGET==2').iloc[:,i].isnull().sum())
            nm=data.iloc[:,i].name
            na_values.loc[nm]=temp
    return(na_values)

#def na_val(data):
#    na_values=pd.DataFrame(columns=[['null_sum','null_in_0','null_in_1', 'null_in_2']])
#    for i in range(0,data.shape[1]):
#        temp=[]
#        if data.iloc[:,i].isnull().sum()>0:
#            temp.append(data.iloc[:,i].isnull().sum())
#            temp.append(data.query('TARGET==0').iloc[:,i].isnull().sum())
#            temp.append(data.query('TARGET==1').iloc[:,i].isnull().sum())
#            temp.append(data.query('TARGET==2').iloc[:,i].isnull().sum())
#            nm=data.iloc[:,i].name
#            na_values.loc[nm]=temp
#    return(na_values)
    
# функция коррекции заполненных imputer'ом категориальных значений
def corr_cat(data, num_cat):
    ft_list=list(na_val(data).index)
    lst_count=pd.DataFrame(columns=[['feature_count_values']])
    for i in range (na_val(data).shape[0]):
        if len(list(data[ft_list[i]].value_counts()))<= num_cat: 
            lst_count.loc[ft_list[i]]=len(list(data[ft_list[i]].value_counts()))
    data[list(lst_count.index)].astype(float).round()
    return(data)

# модель добивания бедных классов 1
def model_bal_1(X_train, y_train):
    X_res, y_res = SMOTE().fit_resample(X_train, y_train)
    return(X_res,y_res)

# модель добивания бедных классов 2
def model_bal_2(X_train, y_train):
    X_res, y_res = ADASYN().fit_resample(X_train, y_train)
    return(X_res,y_res)
    
# модель добивания бедных классов 3
def model_bal_3(X_train, y_train):
    X_res, y_res = BorderlineSMOTE().fit_resample(X_train, y_train)
    return(X_res,y_res)

## Прочитаем данные и обработаем их

In [3]:
# читаем данные
train=pd.read_csv('contest_train.csv')

# выводим важные параметры
print('Размер train:', train.shape)
print(train.TARGET.value_counts())

# удаляем нулевые/пустые столбцы
train=del_zero(train)

# удаляем все признаки в которых больше 9000 пустых значений
train=del_na(train, 9000)

# удаляем все признаки в которых есть дисбаланс, одно из значений сильно подавляющее
train=del_disbal(train, 17000)


Размер train: (18390, 262)
0    13029
1     4237
2     1124
Name: TARGET, dtype: int64
Пустые столбцы  ['FEATURE_3', 'FEATURE_144', 'FEATURE_249', 'FEATURE_256']  удалены
 Признаки с количеством порпусков > 9000 
             null_sum null_in_0 null_in_1 null_in_2
FEATURE_189    18146     12854      4178      1114
FEATURE_190    12889      9201      2826       862
FEATURE_191    11451      8307      2423       721
FEATURE_192     9752      7040      2081       631
FEATURE_193     9382      6823      1977       582
FEATURE_194    12952      9081      2994       877  удалены
 Признаки с дисбалансом > 17000 
             max_sum max_in_0 max_in_1 max_in_2
FEATURE_2     17338    12226     4005     1107
FEATURE_5     18386    13026     4236     1124
FEATURE_6     18172    12877     4185     1110
FEATURE_20    18349    13003     4225     1121
FEATURE_22    17348    12282     3970     1096
FEATURE_25    17391    12312     3981     1098
FEATURE_27    18358    13004     4231     1123
FEATURE_28

## Заполним NaNы

In [4]:
# заполним порпущенные значения 
train=imputer(train,3)
# скорректируем заполненные категориальные значения
train=corr_cat(train, 7)

## Разобъем наши "train" данные на train и test

In [5]:
X=train.iloc[:,1:-1]
y=train.TARGET
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)
target_names = ['class 0', 'class 1', 'class 2']

## Скомпенсируем дисбаланс классов

In [6]:
X_test, y_test = model_bal_3(X_test, y_test)


## Реализуем модель, сделаем предсказание и оценку.

In [7]:
clf = BernoulliNB(alpha = 1)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print(classification_report(y_test, pred, target_names=target_names))
print('F_macro мера не тестовом поднаборе:',f1_score(y_test, pred, average='macro'))


              precision    recall  f1-score   support

     class 0       0.55      0.57      0.56      3840
     class 1       0.49      0.44      0.46      3840
     class 2       0.65      0.69      0.67      3840

    accuracy                           0.57     11520
   macro avg       0.56      0.57      0.56     11520
weighted avg       0.56      0.57      0.56     11520

F_macro мера не тестовом поднаборе: 0.5639040526289887


## Поработаем с набором test

In [8]:
test=pd.read_csv('contest_test.csv')
# заполним порпущенные значения 
test_copy=imputer(test,3)
# скорректируем заполненные категориальные значения
test_copy=corr_cat(test_copy, 7)

In [9]:
temp=train.iloc[:,:-1].copy()
test_copy=test_copy[list(temp.columns)].copy()
pred=clf.predict(test_copy.iloc[:,1:])

In [10]:
result=pd.DataFrame(columns=['ID','TARGET'])
result['ID']=test['ID']
result['TARGET']=pred.astype(int)
result.to_csv('contest_answer.csv', index=False)